## Creating the Environment

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import time

# Parameters
grid_size = 20
discount_factor = 0.9
epsilon = 0.1
alpha = 0.1
theta = 0.0001
num_episodes = 10

In [17]:
# grid with obstacles
grid = np.zeros((grid_size, grid_size))  # Empty grid
obstacles = np.random.choice([0, 1], size=(grid_size, grid_size), p=[0.8, 0.2])  # 20% obstacles
grid += obstacles
print(grid)

[[1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.

In [18]:
# Start and Goal Positions
start = (np.random.randint(0, grid_size), np.random.randint(0, grid_size))
goal = (np.random.randint(0, grid_size), np.random.randint(0, grid_size))

In [19]:
# Set the reward grid
reward_grid = np.zeros_like(grid)
reward_grid[goal] = 100  # Goal reward
reward_grid[grid == 1] = -10  # Obstacle penalty

In [20]:
reward_grid

array([[-10.,   0.,   0., -10.,   0.,   0.,   0., -10.,   0.,   0., -10.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [-10.,   0.,   0.,   0.,   0.,   0., -10.,   0.,   0.,   0., -10.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., -10.],
       [  0., -10., -10.,   0.,   0.,   0.,   0.,   0., -10.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [-10.,   0.,   0.,   0.,   0.,   0.,   0., -10.,   0.,   0., -10.,
          0., -10.,   0.,   0.,   0., -10.,   0.,   0.,   0.],
       [-10., -10.,   0.,   0.,   0., -10.,   0., -10.,   0., -10.,   0.,
          0., -10.,   0., -10., -10.,   0.,   0., -10.,   0.],
       [  0.,   0.,   0., -10.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0., -10.,   0.],
       [  0.,   0.,   0., -10.,   0.,   0.,   0.,   0., -10.,   0.,   0.,
          0.,   0.,   0., 100., -10.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0

In [21]:
# Action space: up, down, left, right, stay
actions = [(0, 1), (0, -1), (1, 0), (-1, 0), (0, 0)]  # (x, y) movements

### MDP Based Approach

In [22]:
# Helper function
def get_next_state(state, action):
    x, y = state
    dx, dy = action
    next_state = (min(max(x + dx, 0), grid_size - 1), min(max(y + dy, 0), grid_size - 1))  # Boundaries
    if grid[next_state] == 1:  # Hit an obstacle
        return state
    return next_state

In [24]:
# Value Iteration Algorithm
def value_iteration():
    V = np.zeros((grid_size, grid_size))
    iterations = 0
    start_time = time.time()

    while True:
        delta = 0
        for x in range(grid_size):
            for y in range(grid_size):
                state = (x, y)
                if state == goal:
                    continue
                v = V[state]
                action_values = []
                for i, action in enumerate(actions):
                    next_state = get_next_state(state, action)
                    reward = reward_grid[next_state]
                    action_values.append(reward + discount_factor * V[next_state])
                V[state] = max(action_values)
                delta = max(delta, abs(v - V[state]))
        iterations += 1
        if delta < theta:
            break

    total_time = time.time() - start_time
    return V, iterations, total_time

In [25]:
V_mdp, iterations_mdp, time_mdp = value_iteration()
print(f"MDP: Converged in {iterations_mdp} iterations, Time taken: {time_mdp:.2f} seconds")

MDP: Converged in 21 iterations, Time taken: 0.15 seconds


### Multi-Armed Bandit (Epsilon-Greedy) Approach

In [26]:
# Q-Table for Multi-Armed Bandit approach
Q = np.zeros((grid_size, grid_size, len(actions)))

# Epsilon-greedy action selection
def epsilon_greedy_action(state):
    if np.random.rand() < epsilon:
        return np.random.randint(len(actions))  # Explore random action
    else:
        return np.argmax(Q[state[0], state[1], :])  # Exploit best action

In [39]:
def run_bandit():
    start_time = time.time()

    # Limit the number of steps per episode to prevent infinite loops
    max_steps_per_episode = 100
    for episode in range(num_episodes):
        state = start
        step = 0
        while state != goal and step < max_steps_per_episode:
            action = epsilon_greedy_action(state)
            next_state = get_next_state(state, actions[action])
            reward = reward_grid[next_state]
            best_next_action = np.argmax(Q[next_state[0], next_state[1], :])

            # Q-Learning update
            Q[state[0], state[1], action] += alpha * (reward + discount_factor * Q[next_state[0], next_state[1], best_next_action] - Q[state[0], state[1], action])
            state = next_state
            step += 1

    total_time = time.time() - start_time
    return total_time

In [40]:
time_bandit = run_bandit()
print(f"Bandit: Time taken: {time_bandit:.2f} seconds")

Bandit: Time taken: 90.28 seconds
